# Recurrent Neural Networks (RNNs)

## What is an RNN?
A Recurrent Neural Network (RNN) is a type of neural network specifically designed to handle sequential data, like text, speech, or time-series data. \
Unlike MLPs, RNNs have `Memory` which allows it to use information from past inputs to influence the current output. \
An MLP is memoryless. It processes each input independently, making it unsuitable for tasks where context and order are important.

RNNs have three key elements:
1. The Input: This is the current item in the sequence.
2. The Hidden State: This is the memory of the network. It is a vector that holds information about previous steps in the sequence.
3. The Output: This is the prediction at the current step.

Throughout all of this, the same three weight matrices are used ($W_{xh}, W_{hh}, \text{and }W_{hy}$).
## Breaking Down the Weight Matrices

Let's break down each of the weight matrices you see in an RNN diagram:

* **$W_{xh}$ (Input-to-Hidden):** This matrix is used to process the current input ($x_t$). It transforms the input data into a vector that can be combined with the hidden state. Think of it as the "new information" gate.
<br>
* **$W_{hh}$ (Hidden-to-Hidden):** This matrix is used to process the previous hidden state ($h_{t-1}$). It transforms the "memory" from the last step so it can be combined with the current input. This is the "old memory" gate.
<br>
* **$W_{yh}$ (Hidden-to-Output):** This matrix takes the current hidden state ($h_t$) and produces the final output ($y_t$) for that step. It's the part of the network that makes a prediction.

***

## How They Are Used to Predict

At each step in the sequence, the RNN calculates a new hidden state and a new output. The process looks like this:

1.  The current input ($x_t$) is multiplied by the **$W_{xh}$** matrix.
2.  The previous hidden state ($h_{t-1}$) is multiplied by the **$W_{hh}$** matrix.
3.  The results of these two operations are summed, and then a non-linear activation function (like `tanh` or `ReLU`) is applied. This produces the new hidden state, $h_t$.
4.  The new hidden state ($h_t$) is then multiplied by the **$W_{yh}$** matrix to produce the output for that step, $y_t$.

The magic is that the *same* **$W_{xh}$**, **$W_{hh}$**, and **$W_{yh}$** matrices are used for every single step of the sequence, which is what allows the RNN to learn patterns in sequential data.

## Backpropagation Through Time (BPTT)

RNNs use a specific form of backpropagation called **Backpropagation Through Time (BPTT)**. The core idea is the same as standard backpropagation, but adapted for sequences:

1.  The network makes a forward pass through the *entire sequence*, calculating the hidden states and outputs for each step.
2.  The loss is calculated at the end of the sequence (or at each step, depending on the task).
3.  The gradients are then propagated backward through the network, but also **backward through time**, from the final step of the sequence to the first.
4.  The optimizer uses these accumulated gradients to update the shared weight matrices (**$W_{xh}$**, **$W_{hh}$**, and **$W_{yh}$**).

***

The key thing to remember is that because the **same weights** are used at every time step, the gradients from all steps in the sequence are **summed up** before the optimizer updates the weights. This is what allows the network to learn how its parameters affect the output across the entire sequence.

## Dataset
A great public dataset for this is the IMDB movie review dataset, which contains thousands of movie reviews labeled as either positive or negative. We can use our RNN to classify the sentiment of a review. \
However, since this is real data, we'll need to do some more robust preprocessing. We'll need to:

1. Tokenize the reviews into individual words.

2. Create a vocabulary of all unique words.

3. Handle sentences of different lengths (since reviews aren't all the same length).

4. Convert words into embeddings to feed into our RNN.

**Disclaimer**: In a real-world scenario with a large text dataset, you would perform extensive preprocessing. This would include creating a vocabulary and using word embeddings to represent your words as vectors. For this simplified example, we'll use a dummy tensor to represent our data.

In [9]:
import torch
import torch.optim as optim
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        
        self.hidden_size = hidden_size
        
        # The RNN layer itself
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        
        # The final linear layer to output the prediction
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # The RNN layer returns both the output and the hidden state
        rnn_out, hidden_state = self.rnn(x)
        
        # We'll use the output of the last step to make our final prediction
        prediction = self.fc(rnn_out[:, -1, :])
        
        return prediction

## "Data" Prep

In [10]:
# We'll have a batch of 2 "sentences" (sequences of word vectors)
# Each sentence has a sequence length of 5 and a word vector size of 10
input_data = torch.randn(2, 5, 10) 
labels = torch.tensor([1, 0]).float().unsqueeze(1) # 1 for positive, 0 for negative

In [11]:
## Training Loop
model = SimpleRNN(input_size=10, hidden_size=20, output_size=1)
criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. The Training Loop
epochs = 5
for epoch in range(epochs):
    # Zero the gradients
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(input_data)
    
    # Compute the loss
    loss = criterion(outputs, labels)
    
    # Backpropagation
    loss.backward()
    
    # Update the weights
    optimizer.step()
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/5], Loss: 0.7645
Epoch [2/5], Loss: 0.7104
Epoch [3/5], Loss: 0.6583
Epoch [4/5], Loss: 0.6083
Epoch [5/5], Loss: 0.5605


# Limitations of RNNs
The two main limitations of RNNs are the **Vanishing Gradient Problem** and the **Exploding Gradient Problem**.
1. **Vanishing Gradient Problem**: In a simple RNN, gradients are used to update the network's weights during backpropagation. As these gradients are propagated backward through time, they can become smaller and smaller with each step. This makes the gradient in earlier layer become close to zero. 
2. **Exploding Gradient Problem**: This is the opposite of the vanishing gradient problem. It occurs when the gradients become extremely large and unstable as they are propagated backward through time. 

## Example of the Vanishing Gradient Problem in RNNs
Imagine a very simple, single-neuron RNN with a single recurrent weight, **$W_{hh} = 0.5$**. The gradient of the loss with respect to this weight is calculated by repeatedly multiplying the error from the output back through the network's layers and through time.

For our simple case, let's say the gradient at the final step (t=5) is `1.0`.

***

**Step 5:** The gradient is `1.0`. To get the gradient for step 4, we multiply by the weight:
* Gradient at t=4: $1.0 \times W_{hh} = 1.0 \times 0.5 = 0.5$

**Step 4:** The gradient is `0.5`. To get the gradient for step 3, we multiply by the weight again:
* Gradient at t=3: $0.5 \times W_{hh} = 0.5 \times 0.5 = 0.25$

**Step 3:** The gradient is `0.25`. To get the gradient for step 2, we multiply by the weight again:
* Gradient at t=2: $0.25 \times W_{hh} = 0.25 \times 0.5 = 0.125$

**Step 2:** The gradient is `0.125`. To get the gradient for step 1, we multiply by the weight again:
* Gradient at t=1: $0.125 \times W_{hh} = 0.125 \times 0.5 = 0.0625$

***

As you can see, the gradient quickly shrinks as it moves backward in time, going from `1.0` all the way down to `0.0625` in just a few steps.

This is a simplified example, but it illustrates the core issue. In a real RNN, the gradient is multiplied by a series of terms, some of which are very small. When these small numbers are multiplied together repeatedly over a long sequence, the final gradient at the beginning of the sequence becomes so small that it is **effectively zero**, and the early layers of the network cannot be updated. This is why the model "forgets" information from earlier in the sequence.

Architectures like LSTMs and GRUs were specifically designed to solve these two problems. They have a more complex internal structure that allows them to better control the flow of gradients and information, giving them a much more robust "memory."